
# **Instructions**:

### 1 - Upload this notebook on [Google Colab](https://colab.research.google.com/) (this notebook need to run on Google Colab).
### 2 - Open the notebook on Google Colab.
### 3 - Change the Google Colab runtime to "GPU" by navigating to "Runtime" > "Change runtime type"
### 4 - Run all cells to start training your model.
### 5 - After the last command finish its execution, copy the file ```yolov4-tiny/training/yolov4-tiny-custom_last.weights``` from your google drive to the cloned project in the same folder as the 4_yolo_opencv_detector notebook.


In [ ]:
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

!ln -s /content/gdrive/My\ Drive/ /mydrive

In [ ]:
!ls /mydrive/yolov4-tiny

In [ ]:
%cd /content/darknet/
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

In [ ]:
!make

In [ ]:
%cd data/
!find -maxdepth 1 -type f -exec rm -rf {} \;
%cd ..

%rm -rf cfg/
%mkdir cfg

In [ ]:
!cp /mydrive/yolov4-tiny/obj.zip ../
!unzip ../obj.zip -d data/

!cp /mydrive/yolov4-tiny/yolov4-tiny-custom.cfg ./cfg
!cp /mydrive/yolov4-tiny/obj.names ./data
!cp /mydrive/yolov4-tiny/obj.data  ./data
!cp /mydrive/yolov4-tiny/process.py ./
!cp /mydrive/yolov4-tiny/yolov4-tiny.conv.29 ./

In [ ]:
import os

def generate_path_files(train_dir, val_dir, output_train="train.txt", output_val="val.txt"):
    """
    Generate text files with absolute paths to images in training and validation directories.
    
    Args:
        train_dir (str): Path to training images directory
        val_dir (str): Path to validation images directory
        output_train (str): Name/path for training output file
        output_val (str): Name/path for validation output file
    """
    # Common image file extensions to consider
    image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff')
    
    def get_image_paths(directory):
        """Helper function to get absolute paths of all images in a directory"""
        image_paths = []
        # Walk through all subdirectories and files
        for root, _, files in os.walk(directory):
            for file in files:
                # Check if file has an image extension
                if file.lower().endswith(image_extensions):
                    # Get absolute path and add to list
                    abs_path = os.path.abspath(os.path.join(root, file))
                    image_paths.append(abs_path)
        return image_paths

    # Get paths for training and validation images
    train_paths = get_image_paths(train_dir)
    val_paths = get_image_paths(val_dir)
    # Ensure train/val directories exist; if not, create them and refresh path lists
    if not os.path.isdir(train_dir):
        os.makedirs(train_dir, exist_ok=True)
        train_paths = get_image_paths(train_dir)
    if not os.path.isdir(val_dir):
        os.makedirs(val_dir, exist_ok=True)
        val_paths = get_image_paths(val_dir)

    # Ensure directories for the output files exist
    for out in (output_train, output_val):
        out_dir = os.path.dirname(out)
        if out_dir and not os.path.exists(out_dir):
            os.makedirs(out_dir, exist_ok=True)
    # Write training paths to file
    with open(output_train, 'w') as f:
        for path in train_paths:
            f.write(f"{path}\n")

    # Write validation paths to file
    with open(output_val, 'w') as f:
        for path in val_paths:
            f.write(f"{path}\n")

    print(f"Successfully generated:")
    print(f"- {output_train} with {len(train_paths)} training images")
    print(f"- {output_val} with {len(val_paths)} validation images")
    
TRAIN_DIRECTORY = "./yolo_custom_dataset/images/train"
VAL_DIRECTORY = "./yolo_custom_dataset/images/val"
    
    # Generate the text files
generate_path_files(TRAIN_DIRECTORY, VAL_DIRECTORY,'./yolov4-tiny/train.txt','./yolov4-tiny/test.txt')    

In [ ]:
!python process.py

In [ ]:
!ls data/

In [ ]:
!./darknet detector train data/obj.data cfg/yolov4-tiny-custom.cfg yolov4-tiny.conv.29 -dont_show